In [4]:
import kagglehub
nguynvitcng21020173_vlsp_2025_data_path = kagglehub.dataset_download('nguynvitcng21020173/vlsp-2025-data')
hung_vimedt5_path = kagglehub.model_download("phmquanghng23020084/vimedt5-300k/other/default")
# minhnvm2307_vitmedt5_finetuned_transformers_default_2_path = kagglehub.model_download('minhnvm2307/vitmedt5-finetuned/Transformers/default/2')
# minhnvm2307_qwen2_5_vlsp_finetuned_transformers_default_1_path = kagglehub.model_download('minhnvm2307/qwen2-5-vlsp-finetuned/Transformers/default/1')

print('Data source import complete.')

Using Colab cache for faster access to the 'vlsp-2025-data' dataset.
Data source import complete.


In [12]:
!mkdir /content/vitmedt5
!cp -r /root/.cache/kagglehub/models/phmquanghng23020084/vimedt5-300k/other/default/1/envit5_final_model_more/* /content/vitmedt5
# !cp -r /root/.cache/kagglehub/datasets/nguynvitcng21020173/vlsp-2025-data/ /content/

mkdir: cannot create directory ‘/content/vitmedt5’: File exists


In [15]:
!pip install -q sacrebleu sacremoses rouge-score

In [16]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from tqdm import tqdm
import sacrebleu
import math
from torch.nn import CrossEntropyLoss
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize
import nltk
from rouge_score import rouge_scorer
import gc

In [14]:
from datasets import Dataset
from transformers import ( AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, )

model_name = "/content/vitmedt5"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to('cuda')

print(type(model))
model.eval()

<class 'transformers.models.t5.modeling_t5.T5ForConditionalGeneration'>


T5ForConditionalGeneration(
  (shared): Embedding(50048, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(50048, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [22]:
# Download NLTK data (quiet)
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)  # thêm để METEOR ổn định hơn
nltk.download('punkt', quiet=True)

class TranslationEvaluator:
    def __init__(self, src_path, tgt_path, model, tokenizer, device="cuda"):
        self.src_path = src_path
        self.tgt_path = tgt_path
        self.tokenizer = tokenizer
        self.device = device
        self.model = model.to(device)
        self.rouge = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
        self.batch_size = 64  # Tối ưu cho 2 GPU

        self.load_data()

    def load_data(self):
        with open(self.src_path, encoding="utf-8") as f:
            self.src_lines = [l.strip() for l in f.readlines()]
        with open(self.tgt_path, encoding="utf-8") as f:
            self.tgt_lines = [l.strip() for l in f.readlines()]

        min_len = min(len(self.src_lines), len(self.tgt_lines))
        self.src_lines = self.src_lines[:min_len]
        self.tgt_lines = self.tgt_lines[:min_len]

    def translate_batch(self, texts, max_len=128):
        outputs = []
        batch_size = self.batch_size
        total = (len(texts) + batch_size - 1) // batch_size

        self.model.eval()
        for i in tqdm(range(0, len(texts), batch_size), total=total, desc="Translating", leave=False):
            batch = texts[i:i + batch_size]
            inputs = self.tokenizer(batch, truncation=True, padding=True, max_length=128, return_tensors="pt").to(self.device)

            with torch.no_grad():
                if isinstance(self.model, nn.DataParallel):
                    generated = self.model.module.generate(**inputs, max_length=max_len, num_beams=5, early_stopping=True)
                else:
                    generated = self.model.generate(**inputs, max_length=max_len, num_beams=5, early_stopping=True)

            decoded = self.tokenizer.batch_decode(generated, skip_special_tokens=True)
            outputs.extend(decoded)

            # Giảm tần suất clear cache để nhanh hơn
            if i % (batch_size * 20) == 0 and i > 0:
                torch.cuda.empty_cache()

        return outputs

    def evaluate_direction(self, src_lines, tgt_lines, direction):
        predictions = self.translate_batch(src_lines)

        # Tính các metric nhanh
        bleu = sacrebleu.corpus_bleu(predictions, [tgt_lines]).score
        ter = sacrebleu.corpus_ter(predictions, [tgt_lines]).score

        # METEOR và ROUGE-L (phải loop, nhưng có tqdm)
        meteor_scores = [meteor_score([word_tokenize(ref)], word_tokenize(pred))
                         for pred, ref in tqdm(zip(predictions, tgt_lines), total=len(predictions), desc="METEOR", leave=False)]
        meteor = sum(meteor_scores) / len(meteor_scores) * 100

        rouge_scores = [self.rouge.score(ref, pred)['rougeL'].fmeasure * 100
                        for pred, ref in tqdm(zip(predictions, tgt_lines), total=len(predictions), desc="ROUGE-L", leave=False)]
        rouge_l = sum(rouge_scores) / len(rouge_scores)

        # In bảng kết quả
        print(f"\n{direction} RESULTS:")
        print(f"BLEU:    {bleu:.2f}")
        print(f"TER:     {ter:.2f}")
        print(f"METEOR:  {meteor:.2f}")
        print(f"ROUGE-L: {rouge_l:.2f}")

        # In 3 ví dụ
        print(f"\nSAMPLE TRANSLATIONS ({direction}):")
        for i in range(min(3, len(predictions))):
            print(f"\n[Sample {i+1}]")
            print(f"SRC:  {src_lines[i]}")
            print(f"PRED: {predictions[i]}")
            print(f"REF:  {tgt_lines[i]}")

        return {
            'bleu': bleu,
            'ter': ter,
            'meteor': meteor,
            'rouge_l': rouge_l,
            'predictions': predictions
        }

    def full_evaluation(self, direction_name="SRC->TGT"):
        return self.evaluate_direction(self.src_lines, self.tgt_lines, direction_name)

In [23]:
import pandas as pd

src_path = "/content/vlsp-2025-data/versions/1/public_test.en.txt"
tgt_path = "/content/vlsp-2025-data/versions/1/public_test.vi.txt"

In [24]:
# Hàm chạy đánh giá 2 chiều (tối ưu, gọn)
def run_bidirectional_evaluation(en_file_path, vi_file_path, model, tokenizer, device="cuda"):
    results = {}

    # EN -> VI
    evaluator = TranslationEvaluator(en_file_path, vi_file_path, model, tokenizer, device)
    results['en_to_vi'] = evaluator.full_evaluation("EN->VI")
    del evaluator
    gc.collect()
    torch.cuda.empty_cache()

    # VI -> EN
    evaluator = TranslationEvaluator(vi_file_path, en_file_path, model, tokenizer, device)
    results['vi_to_en'] = evaluator.full_evaluation("VI->EN")
    del evaluator
    gc.collect()
    torch.cuda.empty_cache()

    return results

In [ ]:
# Chạy đánh giá
results = run_bidirectional_evaluation(
    en_file_path=src_path,   # Thay bằng đường dẫn thực
    vi_file_path=tgt_path, # Thay bằng đường dẫn thực
    model=model,
    tokenizer=tokenizer,
    device="cuda"
)

# In kết quả cuối (nếu cần truy xuất)
print("\nFinal BLEU scores:")
print(f"EN->VI: {results['en_to_vi']['bleu']:.2f}")
print(f"VI->EN: {results['vi_to_en']['bleu']:.2f}")

Translating:  43%|████▎     | 20/47 [04:32<06:12, 13.81s/it]